In [363]:
# import 
import json
import sys

import pandas

In [364]:
# setting

## columns
COLUMNS_CALCED = [
    'average_all', 'average', 'average_exclude_debut', 'highest', 'highest_exclude_debut',
    'over30', 'over50', 'over100', 'over200', 'count_all', 'count' #, 'under30', 'under50', 'under100', 'under200', 
]
COLUMNS = [
    'title', 'date', 'initial', 'kind', # 'r2019u',
    'r2007', 'r2008', 'r2009', 'r2010', 'r2011', 'r2012', 'r2013', 'r2014', 'r2015', 'r2016', 'r2017', 'r2018', 'r2019',
] + COLUMNS_CALCED
RECENT = 'r2019'
BEFORE = 'r2018'
URA = 'r2019'
RANK_INCOMPLETE = ['r2005', 'r2006#1', 'r2006#2']
RANK_ALL = RANK_INCOMPLETE + ['r2007', 'r2008', 'r2009', 'r2010', 'r2011', 'r2012', 'r2013', 'r2014', 'r2015', 'r2016', 'r2017', 'r2018', 'r2019']
RANK = RANK_ALL[3:]
RANK_BEFORE = list(filter(lambda x: x != RECENT, RANK_ALL))

def year_filter(value: str, initial: str, array: list, mode: str = 'ge'):
    if mode not in ['gt', 'ge']:
        raise ValueError(f'invalid mode: {mode}')
    if value in array and initial in array:
        if mode == 'ge':
            return array.index(value) >= array.index(initial)
        else:
            return array.index(value) > array.index(initial)
    elif value in array:
        return True
    else:
        return False

### summary.json

In [365]:
# read json
SUMMARY_PATH = './summary.json'
with open(SUMMARY_PATH, encoding='utf8') as summary_file:
    summary = json.load(summary_file)

In [366]:
# make main dataframe
df = pandas.DataFrame(summary)
df.index = df.no
df['average_all'] = df.average
df = df.drop(columns=['no', 'average'], axis=0)
df.date = pandas.to_datetime(df.date)
df.head()

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019,average_all
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,None,False,False,"[SOME GIRLS -REBEL STREEL IV, CLASSICS, 月光嗜好症]",SOME GIRLS -REBEL STREEL IV,1997-04-25,64.0,r2005,"[SA, OA, OT]",...,96.0,123.0,68.0,169.0,165.0,100.0,134.0,164.0,123.0,121.462
2,青蛾月,None,False,False,"[幻想庭園, étoiles]",幻想庭園,1988-01-25,22.0,r2005,"[SA, OA]",...,72.0,57.0,54.0,87.0,81.0,59.0,146.0,126.0,54.0,80.923
3,マリーゴールド・ガーデン,None,False,False,"[幻想庭園, フラワーチャイルド, 神々の黄昏]",幻想庭園,1988-01-25,128.0,r2005,"[SA, OA, S-2]",...,175.0,141.0,218.0,141.0,238.0,190.0,155.0,164.0,216.0,171.769
4,鏡面界 im Juni,きょうめんかい・イン・ジュニ,False,False,[幻想庭園],幻想庭園,1988-01-25,150.0,r2005,[OA],...,204.0,131.0,174.0,194.0,253.0,244.0,202.0,250.0,260.0,201.077
5,アンジェ・ノワールの祭戯,None,False,False,"[幻想庭園, 神々の黄昏]",幻想庭園,1988-01-25,NaN,r2005,"[SA, OA]",...,81.0,96.0,150.0,106.0,156.0,244.0,187.0,63.0,106.0,125.615


#### works.json

In [367]:
WORKS_PATH = './works.json'
with open(WORKS_PATH, encoding='utf8') as works_file:
    works = json.load(works_file)

In [368]:
dfw = pandas.DataFrame(works)
dfw.date = pandas.to_datetime(dfw.date)
dfw.head()

,title,kind,date
0,SOME GIRLS -REBEL STREEL IV,OT,1997-04-25
1,幻想庭園,OA,1988-01-25
2,フラワーチャイルド,S,1988-07-25
3,恋せよ乙女 -Love story of ZIPANG-,S,1992-07-07
4,月下の一群,OA,1992-12-09


#### info.json

In [369]:
INFO_PATH = './json/info.json'
with open(INFO_PATH, encoding='utf8') as info_file:
    info = json.load(info_file)

lowest = info['max']

* average  
平均順位  
注) データが揃っている2007年以降
* average_exclude_debut  
デビュー年以外の平均順位
* highest  
最高順位
* highest_exclude_debut  
デビュー年以外の最高順位
* over30  
30位を超えた回数
* over50  
50位を超えた回数
* over100  
100位を超えた回数
* over200  
200いを超えた回数
* count_all  
登場回数
* count  
データが揃っている登場回数

In [370]:
df = df.assign(**{key: 0. for key in ['average', 'average_exclude_debut', 'highest', 'highest_exclude_debut', 'lowest']}) \
       .assign(**{key: 0 for key in ['over30', 'over50', 'over100', 'over200', 'count_all', 'count']}) \
       .assign(latest_work=df.works.apply(lambda l: l[-1]))

for no in df.index:
    song = df.loc[no]
    to_mean = list(filter(lambda y: year_filter(y, song.initial, RANK), RANK))
    to_mean_exclude_debut = list(filter(lambda y: year_filter(y, song.initial, RANK, 'gt'), RANK))
    df.average.at[no] = song[to_mean].mean()
    df.average_exclude_debut.at[no] = song[to_mean_exclude_debut].mean()
    df.highest_exclude_debut.at[no] = song[to_mean_exclude_debut].min()
    to_mean_all = list(filter(lambda y: year_filter(y, song.initial, RANK_ALL), RANK_ALL))
    df.highest.at[no] = song[to_mean_all].min()
    df.lowest.at[no] = song[to_mean_all].max()
    df.over30.at[no] = len(list(filter(lambda s: s > 30, song[to_mean_all].values)))
    df.over50.at[no] = len(list(filter(lambda s: s > 50, song[to_mean_all].values)))
    df.over100.at[no] = len(list(filter(lambda s: s > 100, song[to_mean_all].values)))
    df.over200.at[no] = len(list(filter(lambda s: s > 200, song[to_mean_all].values)))
    df.count_all.at[no] = len(to_mean_all)
    df.at[no, 'count'] = len(to_mean)
    # latest work


df.head()
# df.iloc[145]

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,highest,highest_exclude_debut,lowest,over30,over50,over100,over200,count_all,count,latest_work
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,None,False,False,"[SOME GIRLS -REBEL STREEL IV, CLASSICS, 月光嗜好症]",SOME GIRLS -REBEL STREEL IV,1997-04-25,64.0,r2005,"[SA, OA, OT]",...,68.0,68.0,169.0,13,13,9,0,16,13,月光嗜好症
2,青蛾月,None,False,False,"[幻想庭園, étoiles]",幻想庭園,1988-01-25,22.0,r2005,"[SA, OA]",...,54.0,54.0,146.0,13,13,2,0,16,13,étoiles
3,マリーゴールド・ガーデン,None,False,False,"[幻想庭園, フラワーチャイルド, 神々の黄昏]",幻想庭園,1988-01-25,128.0,r2005,"[SA, OA, S-2]",...,141.0,141.0,238.0,13,13,13,3,16,13,神々の黄昏
4,鏡面界 im Juni,きょうめんかい・イン・ジュニ,False,False,[幻想庭園],幻想庭園,1988-01-25,150.0,r2005,[OA],...,131.0,131.0,260.0,13,13,13,6,16,13,幻想庭園
5,アンジェ・ノワールの祭戯,None,False,False,"[幻想庭園, 神々の黄昏]",幻想庭園,1988-01-25,NaN,r2005,"[SA, OA]",...,63.0,63.0,244.0,13,13,9,1,16,13,神々の黄昏


## Hall of Fame

In [371]:
df.query('hall_of_fame')[['title', *RANK_ALL]]

,title,r2005,r2006#1,r2006#2,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
no,,,,,,,,,,,,,,,,,
125,愛と誠,4.0,5.0,8.0,5.0,6.0,4.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
143,亡國覚醒カタルシス,NaN,NaN,3.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,リュウコウセカイ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN


In [372]:
almost = df.query('highest == 1 & ~hall_of_fame & initial != @RECENT')[['title', 'date', *RANK_ALL]]
almost = almost.assign(count=(almost == 1).sum(1))
almost

,title,date,r2005,r2006#1,r2006#2,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019,count
no,,,,,,,,,,,,,,,,,,,
99,戦争と平和,2002-07-24,1.0,1.0,10.0,10.0,7.0,11.0,12.0,12.0,7.0,12.0,14.0,25.0,16.0,15.0,45.0,37.0,2
132,北京LOVERS,2005-06-22,NaN,11.0,18.0,16.0,24.0,17.0,20.0,11.0,11.0,9.0,6.0,8.0,11.0,1.0,13.0,7.0,1
135,聖少女領域,2005-10-26,NaN,4.0,4.0,1.0,2.0,7.0,6.0,3.0,5.0,2.0,3.0,3.0,3.0,10.0,2.0,4.0,1
152,勇侠青春謳,2006-10-25,NaN,NaN,1.0,3.0,4.0,10.0,8.0,7.0,28.0,20.0,24.0,24.0,24.0,18.0,39.0,12.0,1
326,芸術変態論,2018-07-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,6.0,1


In [373]:
print('曲名|発表年|回数|獲得')
print('---|---:|---:|---')
for no in almost.index:
    print(f'{almost.title.at[no]}|{almost.date.at[no].year}|{almost.at[no, "count"]}|')

曲名|発表年|回数|獲得
---|---:|---:|---
戦争と平和|2002|2|
北京LOVERS|2005|1|
聖少女領域|2005|1|
勇侠青春謳|2006|1|
芸術変態論|2018|1|


# MAIN ranking

In [374]:
# target: exclude hall of fame and instrumentals
target = df.query('~hall_of_fame & ~inst').loc[:]
target = target.assign(**{
    'rank_average': target.average.rank(method='min', na_option='bottom'),
    'rank_average_exclude_debut': target.average_exclude_debut.rank(method='min', na_option='bottom'),
    'rank_highest': target.highest.rank(method='min', na_option='bottom'),
    'rank_highest_exclude_debut': target.highest_exclude_debut.rank(method='min', na_option='bottom')
})

target_rank = target[RANK]
target_rank_all = target[RANK_ALL]
target_recent = target[RECENT]
target.head()

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,over50,over100,over200,count_all,count,latest_work,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,None,False,False,"[SOME GIRLS -REBEL STREEL IV, CLASSICS, 月光嗜好症]",SOME GIRLS -REBEL STREEL IV,1997-04-25,64.0,r2005,"[SA, OA, OT]",...,13,9,0,16,13,月光嗜好症,155.0,133.0,193.0,148.0
2,青蛾月,None,False,False,"[幻想庭園, étoiles]",幻想庭園,1988-01-25,22.0,r2005,"[SA, OA]",...,13,2,0,16,13,étoiles,98.0,77.0,168.0,120.0
3,マリーゴールド・ガーデン,None,False,False,"[幻想庭園, フラワーチャイルド, 神々の黄昏]",幻想庭園,1988-01-25,128.0,r2005,"[SA, OA, S-2]",...,13,13,3,16,13,神々の黄昏,226.0,210.0,283.0,254.0
4,鏡面界 im Juni,きょうめんかい・イン・ジュニ,False,False,[幻想庭園],幻想庭園,1988-01-25,150.0,r2005,[OA],...,13,13,6,16,13,幻想庭園,276.0,252.0,269.0,242.0
5,アンジェ・ノワールの祭戯,None,False,False,"[幻想庭園, 神々の黄昏]",幻想庭園,1988-01-25,NaN,r2005,"[SA, OA]",...,13,9,1,16,13,神々の黄昏,163.0,140.0,183.0,137.0


In [375]:
# dfw: works
for y in RANK_ALL:
    if y in RANK:
        dfw[f'{y}_average'] = 0.
    dfw[f'{y}_highest'] = 0.

for idx in dfw.index:
    df_tmp = df[df.works.apply(lambda ws: dfw.at[idx, 'title'] in ws)][RANK_ALL]
    mean = df_tmp.mean()
    for i in mean.index:
        if i not in RANK:
            continue
        dfw.at[idx, f'{i}_average'] = mean.at[i]
    high = df_tmp.min()
    for i in high.index:
        dfw.at[idx, f'{i}_highest'] = high.at[i]

dfw.head()

,title,kind,date,r2005_highest,r2006#1_highest,r2006#2_highest,r2007_average,r2007_highest,r2008_average,r2008_highest,...,r2015_average,r2015_highest,r2016_average,r2016_highest,r2017_average,r2017_highest,r2018_average,r2018_highest,r2019_average,r2019_highest
0,SOME GIRLS -REBEL STREEL IV,OT,1997-04-25,NaN,NaN,NaN,86.000000,86.0,126.000000,126.0,...,165.0,165.0,100.0,100.0,134.0,134.0,164.0,164.0,123.0,123.0
1,幻想庭園,OA,1988-01-25,NaN,NaN,NaN,146.090909,84.0,150.181818,77.0,...,178.2,81.0,184.6,59.0,182.9,130.0,157.4,56.0,185.7,54.0
2,フラワーチャイルド,S,1988-07-25,NaN,NaN,NaN,164.000000,141.0,179.500000,162.0,...,245.5,238.0,217.0,190.0,196.0,155.0,160.0,156.0,238.0,216.0
3,恋せよ乙女 -Love story of ZIPANG-,S,1992-07-07,13.0,13.0,NaN,37.000000,36.0,48.500000,48.0,...,62.0,31.0,138.5,52.0,66.5,20.0,41.0,24.0,88.0,44.0
4,月下の一群,OA,1992-12-09,19.0,13.0,NaN,104.000000,36.0,111.300000,48.0,...,166.6,34.0,187.1,104.0,162.8,54.0,177.7,43.0,150.1,52.0


## recent

In [376]:
target_result = target[['title', 'works', 'debut', 'date', 'initial', 'kind', RECENT, *COLUMNS_CALCED]].astype({RECENT: int}).sort_values(RECENT)

target_result.head()

,title,works,debut,date,initial,kind,r2019,average_all,average,average_exclude_debut,highest,highest_exclude_debut,over30,over50,over100,over200,count_all,count
no,,,,,,,,,,,,,,,,,,
338,BArADiPArADicA,[Fantasia],Fantasia,2019-08-28,r2019,[OA],1,1.000,1.000000,NaN,1.0,NaN,0,0,0,0,1,1
343,自由戀愛,[Fantasia],Fantasia,2019-08-28,r2019,[OA],2,2.000,2.000000,NaN,2.0,NaN,0,0,0,0,1,1
282,Lolicate,"[流行世界, 血と蜜 ～Anthology of Gothic Lolita＆Horror]",流行世界,2014-08-27,r2014,"[OA, CA]",3,6.833,6.833333,7.800000,2.0,3.0,0,0,0,0,6,6
135,聖少女領域,"[聖少女領域, ローゼンメイデン・トロイメント オリジナルサウンドトラック, ローゼンメイデ...",聖少女領域,2005-10-26,r2006#1,"[S-1, OT, ST, CA]",4,3.933,3.923077,3.923077,1.0,1.0,0,0,0,0,15,13
107,月蝕グランギニョル,"[月蝕グランギニョル, Avenger O.S.T., COLLECTION SIMPLE ...",月蝕グランギニョル,2003-11-05,r2005,"[S-1, SA, ST, CA]",5,4.750,4.846154,4.846154,2.0,2.0,0,0,0,0,16,13


In [377]:
print('\\#|曲名|発表年')
print('---:|---|---:')
for no in target_result.head(20).index:
    mark = '**' if target_result.initial.at[no] == RECENT else ''
    print(f'{target_result.r2019.at[no]}|{mark}{target_result.title.at[no]}{mark}|{target_result.date.at[no].year}')

\#|曲名|発表年
---:|---|---:
1|**BArADiPArADicA**|2019
2|**自由戀愛**|2019
3|Lolicate|2014
4|聖少女領域|2005
5|月蝕グランギニョル|2003
6|芸術変態論|2018
7|北京LOVERS|2005
8|未來のイヴ|2003
9|雪華懺悔心中|2008
10|GOD DIVA|2006
11|**少女のための残酷童話**|2019
12|勇侠青春謳|2006
13|今宵、碧い森深く|2006
14|**リアル×ファンタジー**|2019
15|**21世紀新青年**|2019
16|暗黒サイケデリック|2007
17|凶夢伝染|2012
18|卑弥呼外伝|2017
19|青嵐血風録|2007
20|人生美味礼讃|2005


In [378]:
# 初登場かどうか
top20 = pandas.DataFrame({r: [False] * 20 for r in RANK}, index=range(1, 21))
for r in RANK:
    tmp = df[df[r] <= 20].sort_values(r)[['title', r, 'initial']]
    for no in tmp.index:
        top20.at[int(tmp.at[no, r]), r] |= tmp.initial.at[no] == r

top20
# top20.sum()

,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
1,False,False,False,False,False,False,False,True,False,False,False,True,True
2,False,False,True,False,True,True,False,True,False,True,False,False,True
3,False,False,True,False,False,True,True,False,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,True,False,False
5,False,False,True,False,False,False,True,True,True,False,False,False,False
6,False,False,False,False,False,False,True,False,False,True,False,True,False
7,False,False,False,True,False,False,False,False,True,False,False,False,False
8,False,True,False,False,True,True,False,False,False,False,False,False,False
9,False,True,True,False,True,False,False,False,False,False,False,False,False
10,False,False,False,True,False,False,False,False,False,True,False,True,False


In [379]:
targetw_recent = dfw[['title', 'kind', 'date', f'{RECENT}_average', f'{RECENT}_highest']]

targetw_recent.head()

,title,kind,date,r2019_average,r2019_highest
0,SOME GIRLS -REBEL STREEL IV,OT,1997-04-25,123.0,123.0
1,幻想庭園,OA,1988-01-25,185.7,54.0
2,フラワーチャイルド,S,1988-07-25,238.0,216.0
3,恋せよ乙女 -Love story of ZIPANG-,S,1992-07-07,88.0,44.0
4,月下の一群,OA,1992-12-09,150.1,52.0


### debut

In [380]:
debut = target_result.query('initial == @RECENT')[['title', RECENT, 'debut', 'date']]
debut

,title,r2019,debut,date
no,,,,
338,BArADiPArADicA,1,Fantasia,2019-08-28
343,自由戀愛,2,Fantasia,2019-08-28
344,少女のための残酷童話,11,Fantasia,2019-08-28
340,リアル×ファンタジー,14,Fantasia,2019-08-28
339,21世紀新青年,15,Fantasia,2019-08-28
341,Maison de Bonbonnière,21,Fantasia,2019-08-28
348,WALTZ FOR DEBBY,55,Fantasia,2019-08-28
345,恐怖の頭脳改革,61,Fantasia,2019-08-28
337,美しき獣たちの為の,68,『コードギアス 復活のルルーシュ』オリジナルサウンドトラック,2019-02-13


In [381]:
print('\\#|曲名|順位')
print('---:|---|---:')
for idx in range(len(debut)):
    print(f'{idx+1}|{debut.iloc[idx].title}|{debut[RECENT].iat[idx]}')

\#|曲名|順位
---:|---|---:
1|BArADiPArADicA|1
2|自由戀愛|2
3|少女のための残酷童話|11
4|リアル×ファンタジー|14
5|21世紀新青年|15
6|Maison de Bonbonnière|21
7|WALTZ FOR DEBBY|55
8|恐怖の頭脳改革|61
9|美しき獣たちの為の|68
10|地球で|99
11|月下、緑雨幻想|140


In [382]:
print('\\#|lt 20|lt 50|lt 100|over 100')
print('---:|---:|---:|---:|---:')
for y in RANK_ALL[-5:]:
    tmp = df[['title', 'initial', y]][~df[y].isna()].query('initial == @y').sort_values(y)
    print(f'{y[1:]}|{tmp[tmp[y] <= 20].count(0)[y]}|{tmp[tmp[y] <= 50].count(0)[y]}|{tmp[tmp[y] <= 100].count(0)[y]}|{tmp[100 < tmp[y]].count(0)[y]}')


\#|lt 20|lt 50|lt 100|over 100
---:|---:|---:|---:|---:
2015|5|9|11|9
2016|7|8|8|1
2017|2|4|4|0
2018|5|9|9|1
2019|5|6|10|1


In [383]:
# recent 5 originals
or5 = dfw.query('kind == "OA"').tail(5)
or5['initial'] = or5.date.apply(lambda d: d.year)
initials = sorted(set(or5.initial.values))
col = [*[f'r{y}_average' for y in initials], *[f'r{y}_highest' for y in initials]]
or5[['title', *col]]

,title,r2014_average,r2015_average,r2016_average,r2018_average,r2019_average,r2014_highest,r2015_highest,r2016_highest,r2018_highest,r2019_highest
104,流行世界,19.333333,88.1,107.100000,115.222222,152.777778,1.0,1.0,1.0,8.0,3.0
106,快楽のススメ,NaN,44.6,102.100000,143.400000,151.400000,NaN,5.0,26.0,63.0,53.0
109,A級戒厳令,NaN,NaN,25.777778,89.777778,113.777778,NaN,NaN,2.0,9.0,22.0
114,芸術変態論,NaN,NaN,NaN,30.100000,83.300000,NaN,NaN,NaN,1.0,6.0
116,Fantasia,NaN,NaN,NaN,NaN,41.900000,NaN,NaN,NaN,NaN,1.0


In [384]:
target.groupby(by='initial')['title'].count()

initial
r2005      116
r2006#1      6
r2006#2     15
r2007        4
r2008       18
r2009       16
r2010       22
r2011       25
r2012       13
r2013       13
r2014        9
r2015       20
r2016        9
r2017        4
r2018       10
r2019       11
Name: title, dtype: int64

In [385]:
print(f'total : {len(df):>3d}')
print(f'target: {len(target):>3d}')
print(f'debut : {len(debut):>3d}')

total : 348
target: 311
debut :  11


## total

In [457]:
average_top = target.sort_values(['rank_average_exclude_debut', 'date'], ascending=[True, True]) \
                    .query('rank_average_exclude_debut <= 20') \
                    [['title', 'rank_average_exclude_debut', 'average_exclude_debut', 'date']].astype({'rank_average_exclude_debut': int})

average_top

,title,rank_average_exclude_debut,average_exclude_debut,date
no,,,,
135,聖少女領域,1,3.923077,2005-10-26
107,月蝕グランギニョル,2,4.846154,2003-11-05
191,雪華懺悔心中,3,5.800000,2008-08-27
326,芸術変態論,4,6.000000,2018-07-25
282,Lolicate,5,7.800000,2014-08-27
108,未來のイヴ,6,11.076923,2003-11-05
132,北京LOVERS,7,11.846154,2005-06-22
319,まだら恋椿外道,8,13.000000,2016-08-24
145,GOD DIVA,9,13.538462,2006-06-28


In [458]:
print('\\#|曲名|発表年|過去平均|前年までの平均')
print('---:|---|---:|---:|---:')
for no in average_top.index:
    print(f'{average_top.rank_average_exclude_debut.at[no]}|{average_top.title.at[no]}|{average_top.date.at[no].year}|{average_top.average_exclude_debut.at[no]:.3f}|{target[RANK_BEFORE].mean(1).at[no]:.3f}')

\#|曲名|発表年|過去平均|前年までの平均
---:|---|---:|---:|---:
1|聖少女領域|2005|3.923|3.929
2|月蝕グランギニョル|2003|4.846|4.733
3|雪華懺悔心中|2008|5.800|5.200
4|芸術変態論|2018|6.000|1.000
5|Lolicate|2014|7.800|7.600
6|未來のイヴ|2003|11.077|10.933
7|北京LOVERS|2005|11.846|12.571
8|まだら恋椿外道|2016|13.000|6.333
9|GOD DIVA|2006|13.538|13.308
10|凶夢伝染|2012|14.571|12.429
11|卑弥呼外伝|2017|15.000|8.000
12|勇侠青春謳|2006|17.000|16.154
13|戦争と平和|2002|17.154|13.200
14|青嵐血風録|2007|18.273|17.727
15|禁じられた遊び|2004|20.923|17.467
16|KING KNIGHT|2006|21.385|19.538
17|わが﨟たし悪の華|2008|22.300|20.200
18|青空|2018|24.000|6.000
19|胡蝶夢心中|2007|24.818|24.000
20|魅惑劇|2018|25.000|11.000


In [388]:
highest_top = target.sort_values(['rank_highest', 'date'], ascending=[True, True]) \
                    .query('rank_highest <= 20') \
                    [['title', 'rank_highest', 'highest', 'date']].astype({'rank_highest': int, 'highest': int})

highest_top

,title,rank_highest,highest,date
no,,,,
99,戦争と平和,1,1,2002-07-24
132,北京LOVERS,1,1,2005-06-22
135,聖少女領域,1,1,2005-10-26
152,勇侠青春謳,1,1,2006-10-25
326,芸術変態論,1,1,2018-07-25
338,BArADiPArADicA,1,1,2019-08-28
24,嵐ヶ丘,7,2,1993-06-09
107,月蝕グランギニョル,7,2,2003-11-05
121,禁じられた遊び,7,2,2004-10-22


In [389]:
df.r2019

no
1      123.0
2       54.0
3      216.0
4      260.0
5      106.0
       ...  
344     11.0
345     61.0
346     99.0
347      NaN
348     55.0
Name: r2019, Length: 348, dtype: float64

In [390]:
highest_top = target.sort_values(['rank_highest_exclude_debut', 'date'], ascending=[True, True]) \
                    .query('rank_highest_exclude_debut <= 20') \
                    [['title', 'rank_highest_exclude_debut', 'highest_exclude_debut', 'date']].astype({'rank_highest_exclude_debut': int, 'highest_exclude_debut': int})

highest_top

,title,rank_highest_exclude_debut,highest_exclude_debut,date
no,,,,
132,北京LOVERS,1,1,2005-06-22
135,聖少女領域,1,1,2005-10-26
107,月蝕グランギニョル,3,2,2003-11-05
179,逢魔ヶ恋,3,2,2007-10-03
145,GOD DIVA,5,3,2006-06-28
152,勇侠青春謳,5,3,2006-10-25
183,わが﨟たし悪の華,5,3,2008-07-30
282,Lolicate,5,3,2014-08-27
108,未來のイヴ,9,4,2003-11-05


### under

In [391]:
max_rank = target_rank_all.max(1)

In [392]:
target[max_rank <= 10][['title', 'date']].assign(under10=(target_rank_all <= 10).sum(1))

,title,date,under10
no,,,
107,月蝕グランギニョル,2003-11-05,16
135,聖少女領域,2005-10-26,15
191,雪華懺悔心中,2008-08-27,11
326,芸術変態論,2018-07-25,2
338,BArADiPArADicA,2019-08-28,1
343,自由戀愛,2019-08-28,1


In [393]:
print('曲名|発表年|回数')
print('---|---:|---:')
for no in _.index:
    print(f'{_.title.at[no]}|{_.date.at[no].year}|{_.under10.at[no]}')

曲名|発表年|回数
---|---:|---:
月蝕グランギニョル|2003|16
聖少女領域|2005|15
雪華懺悔心中|2008|11
芸術変態論|2018|2
BArADiPArADicA|2019|1
自由戀愛|2019|1


In [394]:
# 初登場から続けてtop 10にいる
N = 2
t = []
len_col = len(df.columns)
for no in df.index:
    idx = df.columns.to_list().index(df.initial.at[no])
    cond = df.at[no, df.initial.at[no]] <= 10
    for i in range(1, N):
        if len_col <= idx + i or df.columns[idx + i] not in RANK_ALL:
            cond = False
            continue
        cond &= df.loc[no].iat[idx + i] <= 10
    if cond:
        t += [df.title.at[no]]
continuous10 = df.query('title in @t')[['title', *RANK_ALL]]
print(len(continuous10))
continuous10

19


,title,r2005,r2006#1,r2006#2,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
no,,,,,,,,,,,,,,,,,
99,戦争と平和,1.0,1.0,10.0,10.0,7.0,11.0,12.0,12.0,7.0,12.0,14.0,25.0,16.0,15.0,45.0,37.0
107,月蝕グランギニョル,5.0,3.0,5.0,4.0,5.0,8.0,4.0,5.0,6.0,7.0,4.0,6.0,4.0,2.0,3.0,5.0
108,未來のイヴ,9.0,7.0,12.0,14.0,19.0,12.0,11.0,13.0,13.0,8.0,8.0,10.0,8.0,16.0,4.0,8.0
112,地獄の季節,6.0,9.0,11.0,13.0,21.0,22.0,37.0,29.0,32.0,17.0,28.0,23.0,25.0,31.0,29.0,26.0
121,禁じられた遊び,2.0,6.0,9.0,6.0,11.0,23.0,27.0,36.0,10.0,21.0,20.0,32.0,19.0,13.0,27.0,27.0
123,阿修羅姫,3.0,10.0,14.0,11.0,25.0,29.0,41.0,42.0,73.0,136.0,79.0,94.0,48.0,48.0,28.0,67.0
125,愛と誠,4.0,5.0,8.0,5.0,6.0,4.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
135,聖少女領域,NaN,4.0,4.0,1.0,2.0,7.0,6.0,3.0,5.0,2.0,3.0,3.0,3.0,10.0,2.0,4.0
143,亡國覚醒カタルシス,NaN,NaN,3.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [395]:
target[(10 < max_rank) & (max_rank <= 20)][['title', 'date']].assign(under20=(target_rank_all <= 20).sum(1))

,title,date,under20
no,,,
108,未來のイヴ,2003-11-05,16
282,Lolicate,2014-08-27,6
322,卑弥呼外伝,2017-03-29,3
339,21世紀新青年,2019-08-28,1
340,リアル×ファンタジー,2019-08-28,1
344,少女のための残酷童話,2019-08-28,1


In [396]:
print('曲名|発表年|回数')
print('---|---:|---:')
for no in _.index:
    print(f'{_.title.at[no]}|{_.date.at[no].year}|{_.under20.at[no]}')

曲名|発表年|回数
---|---:|---:
未來のイヴ|2003|16
Lolicate|2014|6
卑弥呼外伝|2017|3
21世紀新青年|2019|1
リアル×ファンタジー|2019|1
少女のための残酷童話|2019|1


In [397]:
# 初登場から続けてtop 20にいる
N = 16
t = []
len_col = len(df.columns)
for no in df.index:
    idx = df.columns.to_list().index(df.initial.at[no])
    cond = df.at[no, df.initial.at[no]] <= 20
    for i in range(1, N):
        if len_col <= idx + i or df.columns[idx + i] not in RANK_ALL:
            cond = False
            continue
        cond &= df.loc[no].iat[idx + i] <= 20
    if cond:
        t += [df.title.at[no]]
continuous20 = df.query('title in @t')[['title', *RANK_ALL]]
print(len(continuous20))
continuous20

2


,title,r2005,r2006#1,r2006#2,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
no,,,,,,,,,,,,,,,,,
107,月蝕グランギニョル,5.0,3.0,5.0,4.0,5.0,8.0,4.0,5.0,6.0,7.0,4.0,6.0,4.0,2.0,3.0,5.0
108,未來のイヴ,9.0,7.0,12.0,14.0,19.0,12.0,11.0,13.0,13.0,8.0,8.0,10.0,8.0,16.0,4.0,8.0


In [398]:
target[(20 < max_rank) & (max_rank <= 30)][['title', 'date']].assign(under30=(target_rank_all <= 30).sum(1))

,title,date,under30
no,,,
132,北京LOVERS,2005-06-22,15
251,凶夢伝染,2012-01-25,8
319,まだら恋椿外道,2016-08-24,4
331,青空,2018-07-25,2
336,魅惑劇,2018-07-25,2
341,Maison de Bonbonnière,2019-08-28,1


In [399]:
print('曲名|発表年|回数')
print('---|---:|---:')
for no in _.index:
    print(f'{_.title.at[no]}|{_.date.at[no].year}|{_.under30.at[no]}')

曲名|発表年|回数
---|---:|---:
北京LOVERS|2005|15
凶夢伝染|2012|8
まだら恋椿外道|2016|4
青空|2018|2
魅惑劇|2018|2
Maison de Bonbonnière|2019|1


In [400]:
# 初登場から続けてtop 30にいる
N = 2
t = []
len_col = len(df.columns)
for no in df.index:
    initial = df.initial.at[no]
    initial_val = df.at[no, initial]
    idx = df.columns.to_list().index(initial)
    cond = initial_val <= 30 or pandas.isna(initial_val) if initial in RANK_INCOMPLETE else initial_val <= 30
    for i in range(1, N):
        if len_col <= idx + i or df.columns[idx + i] not in RANK_ALL:
            cond = False
            continue
        this_col = df.columns[idx + i]
        this_col_val = df.at[no, this_col]
        cond &= this_col_val <= 30 or pandas.isna(this_col_val) if this_col in RANK_INCOMPLETE else this_col_val <= 30
    if cond:
        t += [df.title.at[no]]
continuous30 = df.query('title in @t')[['title', *RANK_ALL]]
print(len(continuous30))
continuous30

177


,title,r2005,r2006#1,r2006#2,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
no,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,NaN,NaN,NaN,86.0,126.0,116.0,109.0,96.0,123.0,68.0,169.0,165.0,100.0,134.0,164.0,123.0
2,青蛾月,NaN,NaN,NaN,84.0,77.0,73.0,82.0,72.0,57.0,54.0,87.0,81.0,59.0,146.0,126.0,54.0
3,マリーゴールド・ガーデン,NaN,NaN,NaN,141.0,162.0,142.0,150.0,175.0,141.0,218.0,141.0,238.0,190.0,155.0,164.0,216.0
4,鏡面界 im Juni,NaN,NaN,NaN,165.0,185.0,181.0,171.0,204.0,131.0,174.0,194.0,253.0,244.0,202.0,250.0,260.0
5,アンジェ・ノワールの祭戯,NaN,NaN,NaN,109.0,131.0,111.0,93.0,81.0,96.0,150.0,106.0,156.0,244.0,187.0,63.0,106.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,まだら恋椿外道,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,8.0,9.0,22.0
322,卑弥呼外伝,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,12.0,18.0
326,芸術変態論,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,6.0


In [401]:
target[(30 < max_rank) & (max_rank <= 50)][['title', 'date']].assign(under50=(target_rank_all <= 50).sum(1))

,title,date,under50
no,,,
99,戦争と平和,2002-07-24,16
112,地獄の季節,2003-12-03,16
121,禁じられた遊び,2004-10-22,16
145,GOD DIVA,2006-06-28,14
149,KING KNIGHT,2006-09-21,14
152,勇侠青春謳,2006-10-25,14
157,今宵、碧い森深く,2006-12-06,13
168,青嵐血風録,2007-08-22,12
320,悪しき進化,2016-08-24,4


In [402]:
print('曲名|発表年|回数')
print('---|---:|---:')
for no in _.index:
    print(f'{_.title.at[no]}|{_.date.at[no].year}|{_.under50.at[no]}')

曲名|発表年|回数
---|---:|---:
戦争と平和|2002|16
地獄の季節|2003|16
禁じられた遊び|2004|16
GOD DIVA|2006|14
KING KNIGHT|2006|14
勇侠青春謳|2006|14
今宵、碧い森深く|2006|13
青嵐血風録|2007|12
悪しき進化|2016|4
或る修道士の告解|2018|2


In [403]:
# 初登場から続けてtop 50にいる
N = 2
t = []
len_col = len(df.columns)
for no in df.index:
    initial = df.initial.at[no]
    initial_val = df.at[no, initial]
    idx = df.columns.to_list().index(initial)
    cond = initial_val <= 50 or pandas.isna(initial_val) if initial in RANK_INCOMPLETE else initial_val <= 50
    for i in range(1, N):
        if len_col <= idx + i or df.columns[idx + i] not in RANK_ALL:
            cond = False
            continue
        this_col = df.columns[idx + i]
        this_col_val = df.at[no, this_col]
        cond &= this_col_val <= 50 or pandas.isna(this_col_val) if this_col in RANK_INCOMPLETE else this_col_val <= 50
    if cond:
        t += [df.title.at[no]]
continuous50 = df.query('title in @t')[['title', *RANK_ALL]]
print(len(continuous50))
continuous50.tail(20)

208


,title,r2005,r2006#1,r2006#2,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
no,,,,,,,,,,,,,,,,,
264,快恠奇奇,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,33.0,53.0,42.0,68.0,52.0,151.0
265,私の薔薇を喰みなさい,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,39.0,129.0,53.0,62.0,60.0,83.0
268,令嬢薔薇図鑑,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,35.0,57.0,77.0,25.0,44.0,117.0
269,ローズ家の双子達,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,12.0,47.0,45.0,26.0,78.0,106.0
273,少女と水蜜桃,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,23.0,52.0,46.0,28.0,70.0,90.0
274,黒百合隠密カゲキダン,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,10.0,42.0,63.0,103.0,85.0,116.0
279,リュウコウセカイ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN
282,Lolicate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,13.0,12.0,3.0,8.0,3.0
295,Tico Tico no Fubá,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,50.0,61.0,120.0,179.0


In [404]:
target[(50 < max_rank) & (max_rank <= 100)][['title', 'date']].assign(under100=(target_rank_all <= 100).sum(1))

,title,date,under100
no,,,
14,共月亭で逢いましょう,1992-07-07,14
24,嵐ヶ丘,1993-06-09,16
72,コッペリアの柩,1999-12-24,16
115,春葬,2004-06-23,14
126,人生美味礼讃,2005-06-22,16
129,ディレッタントの秘かな愉しみ,2005-06-22,14
158,薔薇獄乙女,2006-12-06,14
166,跪いて足をお嘗め,2007-06-13,12
169,暗黒サイケデリック,2007-08-22,12


In [405]:
col = 2
print('|'.join(['曲名|発表年|回数'] * col))
print('|'.join(['---|---:|---:'] * col))
size = int(pandas.np.ceil(len(_) / col))
l = [[] for __ in range(size)]
for idx in range(len(_)):
    l[idx % size] += [f'{_.title.iat[idx]}|{_.date.iat[idx].year}|{_.under100.iat[idx]}']
for line in l:
    print('|'.join(line))

曲名|発表年|回数|曲名|発表年|回数
---|---:|---:|---|---:|---:
共月亭で逢いましょう|1992|14|刀と鞘|2010|9
嵐ヶ丘|1993|16|RED WALTZ|2012|8
コッペリアの柩|1999|16|少女と水蜜桃|2013|7
春葬|2004|14|L’Amour Looks Something Like You|2015|5
人生美味礼讃|2005|16|放蕩娘と無慈悲な遊戯|2015|5
ディレッタントの秘かな愉しみ|2005|14|薔薇美と百合寧の不思議なホテル|2016|4
薔薇獄乙女|2006|14|絶叫哲学|2016|4
跪いて足をお嘗め|2007|12|少女蜜葬 ～Le sang et le miel|2017|3
暗黒サイケデリック|2007|12|少女昆蟲記|2018|2
胡蝶夢心中|2007|12|極まれリ|2018|2
欲望|2007|12|美しき獣たちの為の|2019|1
暴夜layla幻談|2007|12|恐怖の頭脳改革|2019|1
逢魔ヶ恋|2007|12|地球で|2019|1
わが﨟たし悪の華|2008|11|WALTZ FOR DEBBY|2019|1
Poisoner|2009|10


In [406]:
# 初登場から続けてtop 100にいる
N = 12
t = []
len_col = len(df.columns)
for no in df.index:
    initial = df.initial.at[no]
    initial_val = df.at[no, initial]
    idx = df.columns.to_list().index(initial)
    cond = initial_val <= 100 or pandas.isna(initial_val) if initial in RANK_INCOMPLETE else initial_val <= 100
    for i in range(1, N):
        if len_col <= idx + i or df.columns[idx + i] not in RANK_ALL:
            cond = False
            continue
        this_col = df.columns[idx + i]
        this_col_val = df.at[no, this_col]
        cond &= this_col_val <= 100 or pandas.isna(this_col_val) if this_col in RANK_INCOMPLETE else this_col_val <= 100
    if cond:
        t += [df.title.at[no]]
continuous100 = df.query('title in @t')[['title', *RANK_ALL]]
print(len(continuous100))
continuous100.tail(20)

29


,title,r2005,r2006#1,r2006#2,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
no,,,,,,,,,,,,,,,,,
112,地獄の季節,6.0,9.0,11.0,13.0,21.0,22.0,37.0,29.0,32.0,17.0,28.0,23.0,25.0,31.0,29.0,26.0
115,春葬,10.0,NaN,NaN,48.0,60.0,62.0,39.0,56.0,44.0,55.0,52.0,50.0,49.0,9.0,15.0,39.0
121,禁じられた遊び,2.0,6.0,9.0,6.0,11.0,23.0,27.0,36.0,10.0,21.0,20.0,32.0,19.0,13.0,27.0,27.0
126,人生美味礼讃,18.0,8.0,15.0,12.0,10.0,21.0,28.0,22.0,26.0,23.0,48.0,17.0,61.0,40.0,36.0,20.0
129,ディレッタントの秘かな愉しみ,20.0,NaN,NaN,33.0,42.0,60.0,62.0,66.0,46.0,30.0,57.0,66.0,47.0,45.0,31.0,33.0
130,昭和恋々幻燈館,NaN,NaN,NaN,37.0,41.0,63.0,67.0,56.0,52.0,75.0,95.0,51.0,69.0,41.0,51.0,117.0
132,北京LOVERS,NaN,11.0,18.0,16.0,24.0,17.0,20.0,11.0,11.0,9.0,6.0,8.0,11.0,1.0,13.0,7.0
135,聖少女領域,NaN,4.0,4.0,1.0,2.0,7.0,6.0,3.0,5.0,2.0,3.0,3.0,3.0,10.0,2.0,4.0
145,GOD DIVA,NaN,NaN,7.0,7.0,3.0,6.0,9.0,10.0,29.0,15.0,45.0,14.0,7.0,14.0,7.0,10.0


In [407]:
target[(max_rank <= 100) & (target.works.apply(lambda l: 'Psychedelic Insanity' in l))][['title', 'date']].assign(under100=(target_rank_all <= 100).sum(1))

,title,date,under100
no,,,
168,青嵐血風録,2007-08-22,12
169,暗黒サイケデリック,2007-08-22,12
171,胡蝶夢心中,2007-08-22,12
173,欲望,2007-08-22,12
174,暴夜layla幻談,2007-08-22,12


#### first "over30"

In [408]:
# lowest ever
le = []

In [409]:
target[(30 < target_recent) & (target_recent <= 50)].query('over30 == 1')

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,over50,over100,over200,count_all,count,latest_work,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut
no,,,,,,,,,,,,,,,,,,,,,
333,或る修道士の告解,None,False,False,[芸術変態論],芸術変態論,2018-07-25,NaN,r2018,[OA],...,0,0,0,2,2,芸術変態論,29.0,26.0,97.0,72.0


In [410]:
print('曲名|発表年|順位|過去最低')
print('---|---:|---:|---:')
for no in _.index:
    le += [_.title.at[no]]
    last_val = _[RANK_BEFORE].max(1).at[no]
    last = 'NEW' if pandas.isna(last_val) else int(last_val)
    this_val = int(_[RECENT].at[no])
    this = this_val if lowest[RECENT] > this_val else f"{this_val} (圏外)"
    print(f'{_.title.at[no]}|{_.date.at[no].year}|{this}|{last}')

曲名|発表年|順位|過去最低
---|---:|---:|---:
或る修道士の告解|2018|32|20


#### first "over50"

In [411]:
target[(50 < target_recent) & (target_recent <= 100)].query('over50 == 1')

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,over50,over100,over200,count_all,count,latest_work,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut
no,,,,,,,,,,,,,,,,,,,,,
329,少女昆蟲記,None,False,False,[芸術変態論],芸術変態論,2018-07-25,NaN,r2018,[OA],...,1,0,0,2,2,芸術変態論,44.0,42.0,138.0,113.0
332,極まれリ,None,False,False,[芸術変態論],芸術変態論,2018-07-25,NaN,r2018,[OA],...,1,0,0,2,2,芸術変態論,50.0,85.0,50.0,183.0
337,美しき獣たちの為の,None,False,False,[『コードギアス 復活のルルーシュ』オリジナルサウンドトラック],『コードギアス 復活のルルーシュ』オリジナルサウンドトラック,2019-02-13,NaN,r2019,[ST],...,1,0,0,1,1,『コードギアス 復活のルルーシュ』オリジナルサウンドトラック,81.0,301.0,193.0,301.0
345,恐怖の頭脳改革,None,False,False,[Fantasia],Fantasia,2019-08-28,NaN,r2019,[OA],...,1,0,0,1,1,Fantasia,69.0,301.0,178.0,301.0
346,地球で,テラで,False,False,[Fantasia],Fantasia,2019-08-28,NaN,r2019,[OA],...,1,0,0,1,1,Fantasia,126.0,301.0,233.0,301.0
348,WALTZ FOR DEBBY,None,False,False,[Fantasia],Fantasia,2019-08-28,NaN,r2019,[OA],...,1,0,0,1,1,Fantasia,55.0,301.0,171.0,301.0


In [412]:
print('曲名|発表年|順位|過去最低')
print('---|---:|---:|---:')
for no in _.index:
    le += [_.title.at[no]]
    last_val = _[RANK_BEFORE].max(1).at[no]
    last = 'NEW' if pandas.isna(last_val) else int(last_val)
    this_val = int(_[RECENT].at[no])
    this = this_val if lowest[RECENT] > this_val else f"{this_val} (圏外)"
    print(f'{_.title.at[no]}|{_.date.at[no].year}|{this}|{last}')

曲名|発表年|順位|過去最低
---|---:|---:|---:
少女昆蟲記|2018|51|38
極まれリ|2018|86|10
美しき獣たちの為の|2019|68|NEW
恐怖の頭脳改革|2019|61|NEW
地球で|2019|99|NEW
WALTZ FOR DEBBY|2019|55|NEW


#### first "over100"

In [413]:
target[(100 < target_recent) & (target_recent <= 200)].query('over100 == 1')[['title', 'date', *RANK_ALL]]

,title,date,r2005,r2006#1,r2006#2,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
no,,,,,,,,,,,,,,,,,,
130,昭和恋々幻燈館,2005-06-22,NaN,NaN,NaN,37.0,41.0,63.0,67.0,56.0,52.0,75.0,95.0,51.0,69.0,41.0,51.0,117.0
185,神風,2008-08-27,NaN,NaN,NaN,NaN,NaN,13.0,21.0,14.0,24.0,16.0,27.0,39.0,44.0,24.0,40.0,145.0
261,真夏の憂愁夫人,2012-07-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,56.0,29.0,9.0,33.0,37.0,49.0,101.0
264,快恠奇奇,2013-02-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,33.0,53.0,42.0,68.0,52.0,151.0
268,令嬢薔薇図鑑,2013-09-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,35.0,57.0,77.0,25.0,44.0,117.0
269,ローズ家の双子達,2013-09-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,12.0,47.0,45.0,26.0,78.0,106.0
317,昭和B級下手喰い道,2016-08-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,43.0,55.0,137.0
318,女化生舞楽図,2016-08-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,47.0,63.0,141.0
323,Royal Academy of Gothic Lolita,2017-06-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,77.0,174.0


In [414]:
print('曲名|発表年|順位|過去最低')
print('---|---:|---:|---:')
for no in _.index:
    le += [_.title.at[no]]
    last_val = _[RANK_BEFORE].max(1).at[no]
    last = 'NEW' if pandas.isna(last_val) else int(last_val)
    this_val = int(_[RECENT].at[no])
    this = this_val if lowest[RECENT] > this_val else f"{this_val} (圏外)"
    print(f'{_.title.at[no]}|{_.date.at[no].year}|{this}|{last}')

曲名|発表年|順位|過去最低
---|---:|---:|---:
昭和恋々幻燈館|2005|117|95
神風|2008|145|44
真夏の憂愁夫人|2012|101|56
快恠奇奇|2013|151|68
令嬢薔薇図鑑|2013|117|77
ローズ家の双子達|2013|106|78
昭和B級下手喰い道|2016|137|55
女化生舞楽図|2016|141|63
Royal Academy of Gothic Lolita|2017|174|77
花と龍|2017|192|87
Taolor Tの変身譚|2018|136|47
ヤマトイズム|2018|170|26
タンゴ ダタ|2018|133|35
月下、緑雨幻想|2019|140|NEW


#### first "over200"

In [415]:
target[target_recent > 200].query('over200 == 1')

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,over50,over100,over200,count_all,count,latest_work,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut
no,,,,,,,,,,,,,,,,,,,,,
8,パラソルのある風景,None,False,False,[幻想庭園],幻想庭園,1988-01-25,109.0,r2005,[OA],...,13,13,1,16,13,幻想庭園,236.0,219.0,268.0,241.0
38,冬物語,None,False,False,[星と月のソナタ],星と月のソナタ,1995-12-06,86.0,r2005,[OA],...,13,12,1,16,13,星と月のソナタ,187.0,169.0,216.0,179.0
40,ストロベリーパイをお食べ,None,False,False,"[緑山高校甲子園編 オリジナルサウンドトラック盤, jamais vu]",緑山高校甲子園編 オリジナルサウンドトラック盤,1995-12-06,63.0,r2005,"[ST, CA]",...,13,7,1,16,13,jamais vu,143.0,123.0,183.0,137.0
96,schism,None,False,False,[EROTIC & HERETIC],EROTIC & HERETIC,2002-07-24,NaN,r2005,[OA],...,12,5,1,16,13,EROTIC & HERETIC,130.0,105.0,151.0,100.0
181,コトダマ,None,False,False,"[コトダマ, シゴフミ オリジナルサウンドトラック, La Vita Romantica, ...",コトダマ,2008-01-23,NaN,r2008,"[S-1, ST, CA]",...,7,1,1,12,12,"快恠奇奇 ALI PROJECT Ventennale Music, Art Exhibition",71.0,60.0,39.0,72.0
189,血の断章,None,False,False,"[禁書, 血と蜜 ～Anthology of Gothic Lolita＆Horror]",禁書,2008-08-27,73.0,r2009,"[OA, CA]",...,11,9,1,11,11,血と蜜 ～Anthology of Gothic Lolita＆Horror,205.0,189.0,226.0,194.0
202,戦慄の子供たち,None,False,False,"[戦慄の子供たち, Phantom ～Requiem for the Phantom～ O....",戦慄の子供たち,2009-08-19,59.0,r2010,"[S-1, ST, CA]",...,9,4,1,10,10,"快恠奇奇 ALI PROJECT Ventennale Music, Art Exhibition",120.0,110.0,113.0,122.0
209,Animals on the Earth,None,False,False,[Poison],Poison,2009-08-26,50.0,r2010,[OA],...,8,5,1,10,10,Poison,146.0,136.0,132.0,97.0
252,夜見のたそがれの、うつろなる蒼き瞳の。,None,False,False,"[凶夢伝染, 血と蜜 ～Anthology of Gothic Lolita＆Horror]",凶夢伝染,2012-01-25,51.0,r2012,"[S-2, CA]",...,8,4,1,8,8,血と蜜 ～Anthology of Gothic Lolita＆Horror,159.0,142.0,200.0,157.0


In [416]:
print('曲名|発表年|順位|過去最低')
print('---|---:|---:|---:')
for no in _.index:
    le += [_.title.at[no]]
    last_val = _[RANK_BEFORE].max(1).at[no]
    last = 'NEW' if pandas.isna(last_val) else int(last_val)
    this_val = int(_[RECENT].at[no])
    this = this_val if lowest[RECENT] > this_val else f"{this_val} (圏外)"
    print(f'{_.title.at[no]}|{_.date.at[no].year}|{this}|{last}')

曲名|発表年|順位|過去最低
---|---:|---:|---:
パラソルのある風景|1988|260 (圏外)|200
冬物語|1995|226|200
ストロベリーパイをお食べ|1995|226|151
schism|2002|226|189
コトダマ|2008|211|83
血の断章|2008|260 (圏外)|190
戦慄の子供たち|2009|201|156
Animals on the Earth|2009|226|175
夜見のたそがれの、うつろなる蒼き瞳の。|2012|211|158
見ぬ友へ|2013|201|138
Femme Fatale|2014|226|154
極楽鳥歌|2014|226|133
禁忌 TABOO|2015|201|142
狩猟令嬢ジビエ日誌|2015|226|164
アタラクシア、そして対峙する自己|2015|211|186
波羅蜜蓮華|2015|226|170
永久戒厳令|2016|226|152
最後の美術館|2016|226|192


### no voted
#### last year

In [417]:
lowest[BEFORE] and target[target[BEFORE] == lowest[BEFORE]]

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,over50,over100,over200,count_all,count,latest_work,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut
no,,,,,,,,,,,,,,,,,,,,,
4,鏡面界 im Juni,きょうめんかい・イン・ジュニ,False,False,[幻想庭園],幻想庭園,1988-01-25,150.0,r2005,[OA],...,13,13,6,16,13,幻想庭園,276.0,252.0,269.0,242.0
20,木洩れ陽のワルツ,None,False,False,[月下の一群],月下の一群,1992-12-09,131.0,r2005,[OA],...,13,13,8,16,13,月下の一群,290.0,272.0,301.0,278.0
22,月光浴,None,False,False,"[月下の一群, 星と月のソナタ, Déjà Vu ～THE ORIGINAL BEST 19...",月下の一群,1992-12-09,179.0,r2005,"[OA, CA]",...,13,13,3,16,13,Déjà Vu ～THE ORIGINAL BEST 1992-1995～,243.0,224.0,257.0,229.0
23,楽園喪失 -Paradise Lost-,None,False,False,"[月下の一群, Romance]",月下の一群,1992-12-09,128.0,r2005,"[SA, OA]",...,13,13,6,16,13,Romance,260.0,240.0,252.0,220.0
32,小夜啼鳥,ナイチンゲール,False,False,[DALI],DALI,1994-02-16,174.0,r2005,[OA],...,13,13,9,16,13,DALI,282.0,264.0,265.0,238.0
45,エンジェル・エッグの作り方,None,False,False,[music tracks from Wish],music tracks from Wish,1997-02-05,132.0,r2005,[ST],...,13,13,4,16,13,music tracks from Wish,266.0,246.0,284.0,256.0
46,Chu Chu,None,False,False,[music tracks from Wish],music tracks from Wish,1997-02-05,150.0,r2005,[ST],...,13,13,11,16,13,music tracks from Wish,304.0,289.0,278.0,249.0
47,Sanctuary,None,False,False,[music tracks from Wish],music tracks from Wish,1997-02-05,181.0,r2005,[ST],...,13,13,11,16,13,music tracks from Wish,309.0,292.0,310.0,293.0
50,Water Drop,None,False,False,"[CLAMP学園探偵団 O.S.T 1, CLAMP学園探偵団 Vocal Collecti...",CLAMP学園探偵団 O.S.T 1,1997-06-21,194.0,r2005,"[ST, OT]",...,13,13,8,16,13,CLAMPAZAR ～クランパザール～,296.0,276.0,288.0,266.0


#### this year

In [418]:
lowest[RECENT] and target[target_recent == lowest[RECENT]]

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,over50,over100,over200,count_all,count,latest_work,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut
no,,,,,,,,,,,,,,,,,,,,,
4,鏡面界 im Juni,きょうめんかい・イン・ジュニ,False,False,[幻想庭園],幻想庭園,1988-01-25,150.0,r2005,[OA],...,13,13,6,16,13,幻想庭園,276.0,252.0,269.0,242.0
8,パラソルのある風景,None,False,False,[幻想庭園],幻想庭園,1988-01-25,109.0,r2005,[OA],...,13,13,1,16,13,幻想庭園,236.0,219.0,268.0,241.0
12,フラワーチャイルド,None,False,False,"[フラワーチャイルド, 幻想庭園, 快恠奇奇 ALI PROJECT Ventennale ...",フラワーチャイルド,1988-07-25,165.0,r2005,"[OA, S-1, CA]",...,13,13,8,16,13,"快恠奇奇 ALI PROJECT Ventennale Music, Art Exhibition",283.0,265.0,272.0,243.0
20,木洩れ陽のワルツ,None,False,False,[月下の一群],月下の一群,1992-12-09,131.0,r2005,[OA],...,13,13,8,16,13,月下の一群,290.0,272.0,301.0,278.0
23,楽園喪失 -Paradise Lost-,None,False,False,"[月下の一群, Romance]",月下の一群,1992-12-09,128.0,r2005,"[SA, OA]",...,13,13,6,16,13,Romance,260.0,240.0,252.0,220.0
31,Virtual Fantasy,None,False,False,[DALI],DALI,1994-02-16,148.0,r2005,[OA],...,13,13,5,16,13,DALI,257.0,237.0,259.0,231.0
44,夢のあとに ～après un rêve～,None,False,False,"[Wish, music tracks from Wish, CLAMPAZAR ～クランパ...",Wish,1996-12-24,162.0,r2005,"[S-2, ST, OT, CA]",...,13,13,7,16,13,COLLECTION SIMPLE PLUS,284.0,268.0,300.0,276.0
45,エンジェル・エッグの作り方,None,False,False,[music tracks from Wish],music tracks from Wish,1997-02-05,132.0,r2005,[ST],...,13,13,4,16,13,music tracks from Wish,266.0,246.0,284.0,256.0
46,Chu Chu,None,False,False,[music tracks from Wish],music tracks from Wish,1997-02-05,150.0,r2005,[ST],...,13,13,11,16,13,music tracks from Wish,304.0,289.0,278.0,249.0


### highest ever
(exclude recent year)

In [419]:
target[target.highest == target_recent].query('initial != @RECENT')[['title', 'date', *RANK_ALL]]

,title,date,r2005,r2006#1,r2006#2,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
no,,,,,,,,,,,,,,,,,,
2,青蛾月,1988-01-25,NaN,NaN,NaN,84.0,77.0,73.0,82.0,72.0,57.0,54.0,87.0,81.0,59.0,146.0,126.0,54.0
102,Nostalgia,2002-07-24,NaN,NaN,NaN,151.0,145.0,114.0,135.0,128.0,134.0,110.0,142.0,172.0,131.0,199.0,133.0,77.0
157,今宵、碧い森深く,2006-12-06,NaN,NaN,NaN,23.0,23.0,27.0,30.0,20.0,38.0,32.0,32.0,30.0,27.0,29.0,16.0,13.0
292,修道院の廃庭にて,2015-01-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167.0,225.0,161.0,214.0,101.0


In [420]:
print('曲名|発表年|順位|過去最高')
print('---|---:|---:|---:')
for no in _.index:
        print(f'{_.title.at[no]}|{_.date.at[no].year}|{int(_.at[no, RECENT])}|{int(_[RANK_BEFORE].min(1).at[no])}')

曲名|発表年|順位|過去最高
---|---:|---:|---:
青蛾月|1988|54|54
Nostalgia|2002|77|110
今宵、碧い森深く|2006|13|16
修道院の廃庭にて|2015|101|161


### lowest ever
(voted in recent year)

In [421]:
target[(target.lowest == target_recent) & (target_recent != lowest[RECENT])].query('initial != @RECENT')[['title', 'date', *RANK_ALL]]

,title,date,r2005,r2006#1,r2006#2,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
no,,,,,,,,,,,,,,,,,,
27,エスカルゴ嬉遊曲,1994-01-19,NaN,NaN,NaN,56.0,66.0,91.0,109.0,155.0,194.0,161.0,188.0,215.0,216.0,213.0,200.0,226.0
38,冬物語,1995-12-06,NaN,NaN,NaN,135.0,128.0,135.0,155.0,158.0,150.0,155.0,130.0,200.0,84.0,108.0,103.0,226.0
40,ストロベリーパイをお食べ,1995-12-06,NaN,NaN,NaN,68.0,63.0,74.0,81.0,89.0,127.0,82.0,150.0,115.0,136.0,118.0,151.0,226.0
96,schism,2002-07-24,NaN,NaN,NaN,46.0,74.0,71.0,104.0,97.0,104.0,93.0,76.0,189.0,115.0,59.0,57.0,226.0
98,時の森のソワレ,2002-07-24,17.0,NaN,NaN,66.0,88.0,86.0,85.0,84.0,94.0,123.0,109.0,106.0,138.0,146.0,136.0,159.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,青空,2018-07-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,24.0
332,極まれリ,2018-07-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,86.0
333,或る修道士の告解,2018-07-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,32.0


In [422]:
col = 2
tmp = _.query('title not in @le')
size = int(pandas.np.ceil(len(tmp) / col))
l = [[] for __ in range(size)]
for idx in range(len(tmp)):
    l[idx % size] += [f'{tmp.title.iat[idx]}|{tmp.date.iat[idx].year}|{int(tmp[RECENT].iat[idx])}|{int(tmp[RANK_BEFORE].max(1).iat[idx])}']

print('|'.join(['曲名|発表年|順位|過去最低'] * col))
print('|'.join(['---|---:|---:|---:'] * col))
for line in l:
    print('|'.join(line))

曲名|発表年|順位|過去最低|曲名|発表年|順位|過去最低
---|---:|---:|---:|---|---:|---:|---:
エスカルゴ嬉遊曲|1994|226|216|黒百合隠密カゲキダン|2013|116|103
時の森のソワレ|2002|159|146|赤い蝋燭と金魚|2014|179|117
百合と夜鶯|2005|192|175|海賊皇女|2014|226|225
神の雪|2005|192|144|形而上的な、蝶になる|2014|170|141
In The World|2006|226|219|遊蝶花ワルツ|2015|226|221
名なしの森|2007|226|219|Tico Tico no Fubá|2015|179|120
CYBER DEVILS|2007|226|218|BAR酔芙蓉へどうぞ|2015|192|167
鬼帝の剣|2008|125|102|緋ノ糸輪廻ノGEMINI|2015|166|147
吾君想う故に吾在り生き霊となりて|2009|226|201|まだら恋椿外道|2016|22|9
Hell's Maria|2009|192|179|卑弥呼外伝|2017|18|12
処女懐胎、あるいは白骨塔より少女達は飛翔する|2009|151|109|少女蜜葬 ～Le sang et le miel|2017|94|76
百合の日々は追憶の中に潜み薫る|2010|226|202|芸術変態論|2018|6|1
亂世エロイカ|2010|133|111|眼球ロマン|2018|170|107
平成日本残酷物語|2010|137|126|青空|2018|24|6
真偽贋物遊覧会|2012|226|219|魅惑劇|2018|25|11
少女と水蜜桃|2013|90|70


### diff

In [423]:
diff = target.query('initial != @RECENT')
diff = diff.assign(diff=diff[BEFORE] - diff[RECENT])

### up

In [452]:
diff.sort_values('diff', ascending=False)[['title', 'date', 'diff', *RANK_ALL]].head(10)

,title,date,diff,r2005,r2006#1,r2006#2,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
no,,,,,,,,,,,,,,,,,,,
77,少女貴族,2001-04-25,134.0,NaN,16.0,NaN,24.0,26.0,36.0,52.0,48.0,47.0,78.0,107.0,69.0,88.0,199.0,192.0,58.0
262,天譴と超克,2012-07-18,133.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,79.0,68.0,154.0,146.0,127.0,250.0,117.0
15,メガロポリス・アリス -MEGALOPOLIS ALICE-,1992-12-09,131.0,NaN,NaN,NaN,49.0,59.0,72.0,128.0,158.0,89.0,129.0,156.0,150.0,201.0,219.0,209.0,78.0
270,隼の白バラ,2013-09-11,125.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0,117.0,175.0,119.0,111.0,250.0,125.0
133,鎮魂頌,2005-06-22,125.0,NaN,NaN,19.0,30.0,68.0,57.0,44.0,65.0,65.0,64.0,113.0,119.0,70.0,65.0,221.0,96.0
292,修道院の廃庭にて,2015-01-21,113.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167.0,225.0,161.0,214.0,101.0
33,星降る夜の天文学 ～BEDSIDE ASTRONOMY～,1994-02-16,107.0,NaN,NaN,NaN,141.0,122.0,152.0,104.0,158.0,92.0,115.0,102.0,220.0,86.0,237.0,221.0,114.0
62,Halation,1998-11-21,91.0,NaN,NaN,NaN,106.0,150.0,123.0,192.0,188.0,204.0,136.0,99.0,253.0,225.0,187.0,221.0,130.0
254,贋作師,2012-07-18,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,26.0,16.0,90.0,119.0,39.0,138.0,48.0


In [425]:
print('曲名|発表年|順位|差')
print('---|---:|---:|---:')
for no in _.index:
    print(f'{_.title.at[no]}|{_.date.at[no].year}|{int(_[RECENT].at[no])}|{int(_.at[no, "diff"])}')

曲名|発表年|順位|差
---|---:|---:|---:
少女貴族|2001|58|134
天譴と超克|2012|117|133
メガロポリス・アリス -MEGALOPOLIS ALICE-|1992|78|131
隼の白バラ|2013|125|125
鎮魂頌|2005|96|125
修道院の廃庭にて|2015|101|113
星降る夜の天文学 ～BEDSIDE ASTRONOMY～|1994|114|107
Halation|1998|130|91
贋作師|2012|48|90
ビアンカ|1992|133|88


### down

In [453]:
diff.sort_values('diff')[['title', 'date', 'diff', *RANK_ALL]].head(10)

,title,date,diff,r2005,r2006#1,r2006#2,r2007,r2008,r2009,r2010,r2011,r2012,r2013,r2014,r2015,r2016,r2017,r2018,r2019
no,,,,,,,,,,,,,,,,,,,
96,schism,2002-07-24,-169.0,NaN,NaN,NaN,46.0,74.0,71.0,104.0,97.0,104.0,93.0,76.0,189.0,115.0,59.0,57.0,226.0
328,ヤマトイズム,2018-07-25,-144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,170.0
272,朗読する女中と小さな令嬢,2013-09-11,-138.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.0,147.0,243.0,225.0,213.0,122.0,260.0
181,コトダマ,2008-01-23,-128.0,NaN,NaN,NaN,NaN,8.0,32.0,35.0,38.0,53.0,51.0,62.0,44.0,56.0,72.0,83.0,211.0
38,冬物語,1995-12-06,-123.0,NaN,NaN,NaN,135.0,128.0,135.0,155.0,158.0,150.0,155.0,130.0,200.0,84.0,108.0,103.0,226.0
201,吾君想う故に吾在り生き霊となりて,2009-05-27,-122.0,NaN,NaN,NaN,NaN,NaN,NaN,70.0,128.0,155.0,146.0,156.0,175.0,201.0,169.0,104.0,226.0
170,CYBER DEVILS,2007-08-22,-121.0,NaN,NaN,NaN,NaN,45.0,66.0,104.0,117.0,160.0,218.0,204.0,167.0,158.0,159.0,105.0,226.0
104,プラトニック,2003-04-23,-112.0,NaN,NaN,NaN,133.0,119.0,154.0,150.0,175.0,185.0,189.0,204.0,146.0,158.0,94.0,148.0,260.0
325,花と龍,2017-09-27,-105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,87.0,192.0


In [455]:
print('曲名|発表年|順位|差')
print('---|---:|---:|---:')
for no in _.index:
    val = int(_[RECENT].at[no])
    mark = ' (圏外)' if lowest[RECENT] == val else ''
    print(f'{_.title.at[no]}|{_.date.at[no].year}|{val}{mark}|{int(_.at[no, "diff"])}')

曲名|発表年|順位|差
---|---:|---:|---:
schism|2002|226|-169
ヤマトイズム|2018|170|-144
朗読する女中と小さな令嬢|2013|260 (圏外)|-138
コトダマ|2008|211|-128
冬物語|1995|226|-123
吾君想う故に吾在り生き霊となりて|2009|226|-122
CYBER DEVILS|2007|226|-121
プラトニック|2003|260 (圏外)|-112
花と龍|2017|192|-105
神風|2008|145|-105


### top average works
#### S

In [428]:
dfw.query('kind == "S"').sort_values(f'{RECENT}_average')

,title,kind,date,r2005_highest,r2006#1_highest,r2006#2_highest,r2007_average,r2007_highest,r2008_average,r2008_highest,...,r2015_average,r2015_highest,r2016_average,r2016_highest,r2017_average,r2017_highest,r2018_average,r2018_highest,r2019_average,r2019_highest
33,月蝕グランギニョル,S,2003-11-05,5.0,3.0,5.0,9.0,4.0,12.0,5.0,...,8.000000,6.0,6.000000,4.0,9.000000,2.0,3.500000,3.0,6.500000,5.0
111,卑弥呼外伝,S,2017-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.000000,4.0,12.000000,12.0,18.000000,18.0
29,コッペリアの柩 (Hyper Remix Version),S,2001-09-21,7.0,17.0,17.0,22.0,22.0,43.0,43.0,...,38.000000,38.0,58.000000,58.0,85.000000,85.0,78.000000,78.0,42.000000,42.0
56,勇侠青春謳,S,2006-10-25,NaN,NaN,1.0,16.5,3.0,36.0,4.0,...,71.500000,24.0,47.000000,24.0,41.500000,18.0,130.000000,39.0,54.000000,12.0
62,跪いて足をお嘗め,S,2007-06-13,NaN,NaN,NaN,NaN,NaN,33.0,9.0,...,115.500000,56.0,137.000000,30.0,143.000000,49.0,115.000000,30.0,81.000000,38.0
3,恋せよ乙女 -Love story of ZIPANG-,S,1992-07-07,13.0,13.0,NaN,37.0,36.0,48.5,48.0,...,62.000000,31.0,138.500000,52.0,66.500000,20.0,41.000000,24.0,88.000000,44.0
69,わが臈たし悪の華,S,2008-07-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,110.500000,26.0,95.500000,31.0,102.500000,29.0,117.500000,21.0,89.500000,23.0
26,コッペリアの柩,S,2001-05-23,7.0,17.0,17.0,50.0,22.0,70.0,43.0,...,85.000000,38.0,100.000000,58.0,113.000000,85.0,116.000000,78.0,99.500000,42.0
58,薔薇獄乙女,S,2006-12-06,NaN,NaN,2.0,18.0,8.0,51.0,17.0,...,88.000000,37.0,133.000000,41.0,123.500000,34.0,102.500000,61.0,101.000000,35.0
11,星月夜 ～ルシファー第四楽章～,S,1996-08-01,NaN,NaN,NaN,138.0,89.0,116.5,104.0,...,122.500000,45.0,134.000000,67.0,123.000000,44.0,129.500000,59.0,103.000000,36.0


#### OA, SA, CA

In [429]:
dfw.query('kind =="OA"').sort_values(f'{RECENT}_average')[['title', 'date', f'{BEFORE}_average', f'{RECENT}_average']]

,title,date,r2018_average,r2019_average
116,Fantasia,2019-08-28,NaN,41.900000
43,Dilettante,2005-06-22,93.875000,77.000000
114,芸術変態論,2018-07-25,30.100000,83.300000
63,Psychedelic Insanity,2007-08-22,97.444444,102.888889
31,EROTIC & HERETIC,2002-07-24,109.500000,109.200000
109,A級戒厳令,2016-08-24,89.777778,113.777778
97,贋作師,2012-07-18,136.500000,121.400000
9,星と月のソナタ,1995-12-06,134.300000,135.400000
7,DALI,1994-02-16,141.888889,137.333333
21,Noblerot,1998-11-21,159.700000,139.000000


In [430]:
print('\\#|タイトル|発売年|平均|前年平均')
print('---:|---|---:|---:|---:')
for idx in range(len(_)):
    print(f'{idx+1}|{_.title.iat[idx]}|{_.date.iat[idx].year}|{_[f"{RECENT}_average"].iat[idx]:.3f}|{_[f"{BEFORE}_average"].iat[idx]:.3f}')

\#|タイトル|発売年|平均|前年平均
---:|---|---:|---:|---:
1|Fantasia|2019|41.900|nan
2|Dilettante|2005|77.000|93.875
3|芸術変態論|2018|83.300|30.100
4|Psychedelic Insanity|2007|102.889|97.444
5|EROTIC & HERETIC|2002|109.200|109.500
6|A級戒厳令|2016|113.778|89.778
7|贋作師|2012|121.400|136.500
8|星と月のソナタ|1995|135.400|134.300
9|DALI|1994|137.333|141.889
10|Noblerot|1998|139.000|159.700
11|禁書|2008|146.700|130.600
12|令嬢薔薇図鑑|2013|148.111|130.667
13|Poison|2009|149.778|132.000
14|Aristocracy|2001|149.900|172.500
15|月下の一群|1992|150.100|177.700
16|快楽のススメ|2015|151.400|143.400
17|流行世界|2014|152.778|115.222
18|幻想庭園|1988|185.700|157.400
19|汎新日本主義|2010|205.200|197.600


In [431]:
dfw.query('kind =="SA"').sort_values(f'{RECENT}_average')[['title', 'date', f'{BEFORE}_average', f'{RECENT}_average']]

,title,date,r2018_average,r2019_average
66,Grand Finale,2007-12-12,107.571429,99.428571
32,月光嗜好症,2003-04-23,114.500000,119.250000
57,Romance,2006-12-06,147.111111,141.444444
36,étoiles,2004-06-23,169.333333,170.666667
45,神々の黄昏,2005-12-07,138.777778,184.111111
88,Gothic Opera,2010-03-17,168.111111,188.111111
105,Violetta Operetta,2015-01-21,207.111111,192.000000
95,Les Papillons,2011-06-29,194.888889,212.777778


In [432]:
print('\\#|タイトル|発売年|平均|前年平均')
print('---:|---|---:|---:|---:')
for idx in range(len(_)):
    print(f'{idx+1}|{_.title.iat[idx]}|{_.date.iat[idx].year}|{_[f"{RECENT}_average"].iat[idx]:.3f}|{_[f"{BEFORE}_average"].iat[idx]:.3f}')

\#|タイトル|発売年|平均|前年平均
---:|---|---:|---:|---:
1|Grand Finale|2007|99.429|107.571
2|月光嗜好症|2003|119.250|114.500
3|Romance|2006|141.444|147.111
4|étoiles|2004|170.667|169.333
5|神々の黄昏|2005|184.111|138.778
6|Gothic Opera|2010|188.111|168.111
7|Violetta Operetta|2015|192.000|207.111
8|Les Papillons|2011|212.778|194.889


In [433]:
dfw.query('kind =="CA"').sort_values(f'{RECENT}_average')[['title', 'date', f'{BEFORE}_average', f'{RECENT}_average']]

,title,date,r2018_average,r2019_average
73,桂冠詩人 SINGLE COLLECTION PLUS,2008-12-10,110.230769,94.846154
98,"快恠奇奇 ALI PROJECT Ventennale Music, Art Exhibition",2013-02-20,74.709677,96.677419
113,愛と誠 ～YAMATO＆LOVE×××,2017-09-27,115.580645,115.741935
112,血と蜜 ～Anthology of Gothic Lolita＆Horror,2017-06-21,99.451613,119.774194
60,薔薇架刑,2007-04-04,110.666667,123.583333
52,COLLECTION SIMPLE PLUS,2006-07-26,123.000000,126.500000
49,Déjà Vu ～THE ORIGINAL BEST 1992-1995～,2006-03-08,145.000000,128.357143
87,La Vita Romantica,2010-01-13,120.923077,143.846154
24,jamais vu,2000-08-04,121.400000,146.600000
94,QUEENDOM,2011-05-25,172.636364,179.454545


In [434]:
print('\\#|タイトル|発売年|平均|前年平均')
print('---:|---|---:|---:|---:')
for idx in range(len(_)):
    print(f'{idx+1}|{_.title.iat[idx]}|{_.date.iat[idx].year}|{_[f"{RECENT}_average"].iat[idx]:.3f}|{_[f"{BEFORE}_average"].iat[idx]:.3f}')

\#|タイトル|発売年|平均|前年平均
---:|---|---:|---:|---:
1|桂冠詩人 SINGLE COLLECTION PLUS|2008|94.846|110.231
2|快恠奇奇 ALI PROJECT Ventennale Music, Art Exhibition|2013|96.677|74.710
3|愛と誠 ～YAMATO＆LOVE×××|2017|115.742|115.581
4|血と蜜 ～Anthology of Gothic Lolita＆Horror|2017|119.774|99.452
5|薔薇架刑|2007|123.583|110.667
6|COLLECTION SIMPLE PLUS|2006|126.500|123.000
7|Déjà Vu ～THE ORIGINAL BEST 1992-1995～|2006|128.357|145.000
8|La Vita Romantica|2010|143.846|120.923
9|jamais vu|2000|146.600|121.400
10|QUEENDOM|2011|179.455|172.636
11|絶対音楽|2014|nan|nan


#### I voted

In [435]:
my = ['ナルシス・ノワール', '月蝕グランギニョル', '少女殉血', '未來のイヴ', '地獄の季節', '熱帯性植物園', '遊月恋歌', '戦争と平和', '暗黒天国', '胡蝶夢心中', '六道輪廻サバイバル', '魅惑劇']
voted = target.query('title in @my').assign(pt=1)

for no in voted.index:
    t = voted.title.at[no]
    if my.index(t) == 0:
        voted.pt.at[no] = 3
    elif my.index(t) in [1, 2, 3]:
        voted.pt.at[no] = 2

voted

,title,ruby,inst,hall_of_fame,works,debut,date,r2019u,initial,kind,...,over100,over200,count_all,count,latest_work,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut,pt
no,,,,,,,,,,,,,,,,,,,,,
66,ナルシス・ノワール,None,False,False,"[Noblerot, 月光嗜好症]",Noblerot,1998-11-21,NaN,r2005,"[SA, OA]",...,1,0,16,13,月光嗜好症,47.0,38.0,57.0,46.0,3
94,熱帯性植物園,None,False,False,[EROTIC & HERETIC],EROTIC & HERETIC,2002-07-24,26.0,r2005,[OA],...,7,0,16,13,EROTIC & HERETIC,141.0,122.0,161.0,113.0,1
95,遊月恋歌,None,False,False,"[EROTIC & HERETIC, Romance, 愛と誠 ～YAMATO＆LOVE×××]",EROTIC & HERETIC,2002-07-24,2.0,r2005,"[SA, OA, CA]",...,1,0,16,13,愛と誠 ～YAMATO＆LOVE×××,48.0,39.0,61.0,38.0,1
99,戦争と平和,None,False,False,"[EROTIC & HERETIC, Grand Finale, 愛と誠 ～YAMATO＆L...",EROTIC & HERETIC,2002-07-24,NaN,r2005,"[SA, OA, CA]",...,0,0,16,13,愛と誠 ～YAMATO＆LOVE×××,19.0,13.0,1.0,14.0,1
107,月蝕グランギニョル,None,False,False,"[月蝕グランギニョル, Avenger O.S.T., COLLECTION SIMPLE ...",月蝕グランギニョル,2003-11-05,NaN,r2005,"[S-1, SA, ST, CA]",...,0,0,16,13,"快恠奇奇 ALI PROJECT Ventennale Music, Art Exhibition",5.0,2.0,7.0,3.0,2
108,未來のイヴ,None,False,False,"[月蝕グランギニョル, Avenger O.S.T., COLLECTION SIMPLE ...",月蝕グランギニョル,2003-11-05,NaN,r2005,"[S-2, ST, CA]",...,0,0,16,13,COLLECTION SIMPLE PLUS,10.0,6.0,23.0,9.0,2
111,少女殉血,None,False,False,"[Avenger O.S.T., 桂冠詩人 SINGLE COLLECTION PLUS, ...",Avenger O.S.T.,2003-12-03,4.0,r2005,"[ST, CA]",...,7,1,16,13,愛と誠 ～YAMATO＆LOVE×××,140.0,121.0,53.0,62.0,2
112,地獄の季節,None,False,False,"[Avenger O.S.T., COLLECTION SIMPLE PLUS]",Avenger O.S.T.,2003-12-03,NaN,r2005,"[ST, CA]",...,0,0,16,13,COLLECTION SIMPLE PLUS,27.0,21.0,30.0,28.0,1
164,暗黒天国,None,False,False,"[暗黒天国, かみちゃまかりん オリジナル・サウンドトラック, La Vita Romant...",暗黒天国,2007-05-23,NaN,r2008,"[S-1, ST, CA]",...,1,0,12,12,"快恠奇奇 ALI PROJECT Ventennale Music, Art Exhibition",61.0,52.0,90.0,44.0,1


In [436]:
print('pt|曲名|発表年|順位|前回')
print('---:|---|---:|---:|--:')
for no in voted.sort_values('pt', ascending=False).index:
    print(f'{voted.pt.at[no]}|{voted.title.at[no]}|{voted.date.at[no].year}|{int(voted.at[no, RECENT])}|{int(voted.at[no, BEFORE])}')

pt|曲名|発表年|順位|前回
---:|---|---:|---:|--:
3|ナルシス・ノワール|1998|31|32
2|月蝕グランギニョル|2003|5|3
2|未來のイヴ|2003|8|4
2|少女殉血|2003|71|126
1|熱帯性植物園|2002|100|175
1|遊月恋歌|2002|49|110
1|戦争と平和|2002|37|45
1|地獄の季節|2003|26|29
1|暗黒天国|2007|73|86
1|胡蝶夢心中|2007|43|18
1|六道輪廻サバイバル|2007|94|99
1|魅惑劇|2018|25|11


# URA ranking

In [437]:
# ura ranking
ura = df[df[URA] > 100][COLUMNS]
ura['rank_average'] = ura.average.rank(method='min', na_option='bottom')
ura['rank_average_exclude_debut'] = ura.average_exclude_debut.rank(method='min', na_option='bottom')
ura['rank_highest'] = ura.highest.rank(method='min', na_option='bottom')
ura['rank_highest_exclude_debut'] = ura.highest_exclude_debut.rank(method='min', na_option='bottom')

ura.head()

,title,date,initial,kind,r2007,r2008,r2009,r2010,r2011,r2012,...,over30,over50,over100,over200,count_all,count,rank_average,rank_average_exclude_debut,rank_highest,rank_highest_exclude_debut
no,,,,,,,,,,,,,,,,,,,,,
1,桜の花は狂い咲き,1997-04-25,r2005,"[SA, OA, OT]",86.0,126.0,116.0,109.0,96.0,123.0,...,13,13,9,0,16,13,58.0,46.0,97.0,64.0
3,マリーゴールド・ガーデン,1988-01-25,r2005,"[SA, OA, S-2]",141.0,162.0,142.0,150.0,175.0,141.0,...,13,13,13,3,16,13,126.0,120.0,183.0,164.0
4,鏡面界 im Juni,1988-01-25,r2005,[OA],165.0,185.0,181.0,171.0,204.0,131.0,...,13,13,13,6,16,13,176.0,162.0,169.0,152.0
5,アンジェ・ノワールの祭戯,1988-01-25,r2005,"[SA, OA]",109.0,131.0,111.0,93.0,81.0,96.0,...,13,13,9,1,16,13,65.0,52.0,87.0,55.0
6,紅い睡蓮の午後,1988-01-25,r2005,"[SA, OA]",144.0,176.0,153.0,164.0,188.0,204.0,...,13,13,13,5,16,13,150.0,143.0,178.0,159.0


In [438]:
col = 4
length = len(ura)
row = int(pandas.np.ceil(length/col))

a = [[] for _ in range(col)]
c = 0
for no in ura.index:
    a[int(c // row)] += [ura.title.at[no]]
    c += 1

for i in range(row):
    print('|'.join([a[j][i] if j * row + (i + 1) <= length else '' for j in range(col)]))

桜の花は狂い咲き|病める薔薇|眠れる豹|真偽贋物遊覧会
マリーゴールド・ガーデン|MALICE|血の断章|La vérité
鏡面界 im Juni|à la cuisine|薨|野性双生児
アンジェ・ノワールの祭戯|桂花葬|小さき者への贖罪のためのアリア|逝ける王女の肖像画
紅い睡蓮の午後|閉ざされた画室|鬼帝の剣|真夏の憂愁夫人
少女忌恋歌|un tableau blanc ～絵画旅行～|桂冠詩人|天譴と超克
パラソルのある風景|闇の翼ですべてをつつむ夜のためのアリア|裸々イヴ新世紀|快恠奇奇
硝子天井のうちゅう|après le noir|騎士乙女|Fräulein Rose
幻想庭園|宵待草|吾君想う故に吾在り生き霊となりて|乙女の贖い
フラワーチャイルド|La vie en rose|戦慄の子供たち|令嬢薔薇図鑑
恋せよ乙女 -Love story of ZIPANG-|schism|Hell's Maria|ローズ家の双子達
ビアンカ|時の森のソワレ|処女懐胎、あるいは白骨塔より少女達は飛翔する|隼の白バラ
空宙舞踏会|夢魔の夜、あなたを迎える者がある|お毒味LADY|薔薇娼館
木洩れ陽のワルツ|プラトニック|Animals on the Earth|朗読する女中と小さな令嬢
堕ちて候|Féliciter|上海繚乱ロマンチカ|黒百合隠密カゲキダン
月光浴|MOTHER|世紀末ゲネシス|南無地獄大菩薩
楽園喪失 -Paradise Lost-|繭|この國の向こうに|見ぬ友へ
舞踏会の手帖|Je te veux|輪廻闇妖散華|PANDEMIC
エスカルゴ嬉遊曲|Après un rêve|堕天國宣戦|赤い蝋燭と金魚
サロメティック・ルナティック|Daphné|寶島|Femme Fatale
雪のひとひら|わが恋は月光に散りぬ|月光オペラの天鵞絨の緞帳は上がり|海賊皇女
ダリの宝石店|pastel pure|恋の翼は真珠色の骨を持ち|静謐なる私小説
Virtual Fantasy|あたしがアリスだった頃|私の心は語る薔薇の御胸を葩の御身を|形而上的な、蝶になる
小夜啼鳥|君がため、惜しからざりし命さへ|エルフの娘は地上に降り|極楽鳥歌
星降る夜の天文学 ～BEDSIDE ASTRONOMY～|肉体の悪魔|百合の日々は追憶の中に潜み薫る|遊蝶花ワルツ
オフェリア

In [439]:
average_top = ura.sort_values(['rank_average_exclude_debut', 'date'], ascending=[True, True]) \
                 .query('average_exclude_debut <= 100') \
                 [['title', 'rank_average_exclude_debut', 'average_exclude_debut', 'date']].astype({'rank_average_exclude_debut': int})

average_top

,title,rank_average_exclude_debut,average_exclude_debut,date
no,,,,
185,神風,1,39.400000,2008-08-27
261,真夏の憂愁夫人,2,44.857143,2012-07-18
269,ローズ家の双子達,3,52.333333,2013-09-11
268,令嬢薔薇図鑑,4,59.166667,2013-09-11
130,昭和恋々幻燈館,5,62.692308,2005-06-22
205,処女懐胎、あるいは白骨塔より少女達は飛翔する,6,63.222222,2009-08-26
195,鬼帝の剣,7,64.300000,2008-11-19
264,快恠奇奇,8,66.500000,2013-02-20
181,コトダマ,9,67.000000,2008-01-23


In [440]:
for no in average_top.index:
    print(f'{average_top.rank_average_exclude_debut.at[no]}|{average_top.title.at[no]}|{average_top.average_exclude_debut.at[no]:.2f}')

1|神風|39.40
2|真夏の憂愁夫人|44.86
3|ローズ家の双子達|52.33
4|令嬢薔薇図鑑|59.17
5|昭和恋々幻燈館|62.69
6|処女懐胎、あるいは白骨塔より少女達は飛翔する|63.22
7|鬼帝の剣|64.30
8|快恠奇奇|66.50
9|コトダマ|67.00
10|黒百合隠密カゲキダン|69.83
11|堕天國宣戦|72.67
12|昭和B級下手喰い道|78.33
13|お毒味LADY|79.89
14|女化生舞楽図|83.67
15|亂世エロイカ|86.12
16|ヘテロ失楽園|88.50
17|陸と海と空と|89.25
18|上海繚乱ロマンチカ|90.78
19|恋せよ乙女 -Love story of ZIPANG-|91.85
20|月光夜|96.46
21|平成日本残酷物語|96.62


In [441]:
highest_top = ura.sort_values(['rank_highest_exclude_debut', 'date'], ascending=[True, True]) \
                 .query('highest_exclude_debut <= 50') \
                 [['title', 'rank_highest_exclude_debut', 'highest_exclude_debut', 'highest', 'date']].astype({'rank_highest_exclude_debut': int, 'highest_exclude_debut': int, 'highest': int})

highest_top

,title,rank_highest_exclude_debut,highest_exclude_debut,highest,date
no,,,,,
261,真夏の憂愁夫人,1,9,9,2012-07-18
274,黒百合隠密カゲキダン,2,10,4,2013-09-11
269,ローズ家の双子達,3,12,3,2013-09-11
185,神風,4,14,13,2008-08-27
215,堕天國宣戦,4,14,7,2009-10-21
206,お毒味LADY,6,18,13,2009-08-26
205,処女懐胎、あるいは白骨塔より少女達は飛翔する,7,19,17,2009-08-26
195,鬼帝の剣,8,23,5,2008-11-19
268,令嬢薔薇図鑑,9,25,6,2013-09-11


In [442]:
tmp = ura.loc[:, :]
tmp['under100_mean'] = tmp.under100 / tmp.count_all
tmp.sort_values(['under100_mean', 'date'], ascending=[False, True]) \
   .query('under100_mean >= 0.8') \
   [['title', 'under100', 'under100_mean', 'date']]

AttributeError: 'DataFrame' object has no attribute 'under100'

In [0]:
# 未投票
zero = ['日曜日のシエスタ', 'La vie en rose', 'Féliciter', 'Nocturnal', '寶島', '逢魔蛾城の伯爵', '小夜すみれ ～ある夜の森の小さなお話', '魂ノ代']
ura.query('title in @zero')

In [0]:
ura[ura.kind.apply(lambda k: 'S-1' in k or 'S-2' in k)]

In [0]:
# ura_list[(160 <= ura_list[2018]) & (ura_list[2018] <= 210)]

In [0]:
# len(_)

In [0]:
my_before = ['桜の花は狂い咲き', '乙女の祈り', 'Rose Moon', '少女貴族', '熱帯性植物園', '遊月恋歌', '少女殉血', 'Animals on the Earth', '四神獣飼殺し', 'ALICE同罪イノセント', '陸と海と空と', '緋ノ糸輪廻ノGEMINI']
ura.query('title in @my_before')

In [0]:
ura.title.values

In [0]:
my = ['桜の花は狂い咲き', "Nous Deux C'est Pour La Vie", '月夜のピエレット', '神風', 'Animals on the Earth', '絶國TEMPEST', '四神獣飼殺し', 'La vérité', '真夏の憂愁夫人', '令嬢薔薇図鑑', '陸と海と空と', '緋ノ糸輪廻ノGEMINI']
ura.query('title in @my')

In [0]:
# for idx, i in enumerate(map(lambda line: f'<p>{line}</p>', ura_list.title)):
#     print(i, end='')
#     if idx % 50 == 49:
#         print()

In [0]:
# for idx, i in enumerate(map(lambda line: f'<li>{line}</li>', ura_list.title)):
#     print(i, end='')
#     if idx % 100 == 99:
#         print()